In [2]:
from Neural_Nets.LaengeNetModified.Development.LaengeNetModifiedTorch import LaengeNetModified, LaengeNetModifiedLossFunc
from Neural_Nets.ThermoDataset.Development.ThermoDataset import ThermoDataset
from Utils.PlotHandler.Development.PlotHandler import PlotHandler 
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from torch.optim import Rprop
from Data_Handling.SGTEHandler.Development.SGTEHandler import SGTEHandler
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'Neural_Nets.LaengeNetModified.Development.LaengeNetModifiedTorch'